# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [6]:
# checking your current working directory
print(os.getcwd())
file_path_list = []
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list1 = glob.glob(os.path.join(root,'*'))
    print(file_path_list1)
    file_path_list.append(file_path_list1)

/home/workspace
['/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-08-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-13-events.c

In [7]:
a = file_path_list[0]

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
#for f in file_path_list:
for f in a:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


['Muse', 'Logged In', 'Harper', 'M', '1', 'Barrett', '209.50159', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Supermassive Black Hole (Twilight Soundtrack Version)', '200', '1.54172E+12', '42']
['Beastie Boys', 'Logged In', 'Harper', 'M', '2', 'Barrett', '161.56689', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Lighten Up', '200', '1.54172E+12', '42']
['Shakira', 'Logged In', 'Harper', 'M', '3', 'Barrett', '145.84118', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Pienso En Ti', '200', '1.54172E+12', '42']
['Selena', 'Logged In', 'Harper', 'M', '4', 'Barrett', '172.66893', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSong', '1.54069E+12', '275', 'Amor Prohibido', '200', '1.54172E+12', '42']
['Kid Cudi Vs Crookers', 'Logged In', 'Harper', 'M', '5', 'Barrett', '162.97751', 'paid', 'New York-Newark-Jersey City, NY-NJ-PA', 'PUT', 'NextSon

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))


6821


In [10]:
df = pd.read_csv('/home/workspace/event_datafile_new.csv')
df.count()
df.columns
#Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
#       'level', 'location', 'sessionId', 'song', 'userId'],
#      dtype='object')
df.dtypes
#artist            object
#firstName         object
#gender            object
#itemInSession      int64
#lastName          object
#length           float64
#level             object
#location          object
#sessionId          int64
#song              object
#userId             int64

#artist text,firstName text,gender text,itemInSession int,lastName int,
#length float, level text,location text,sessionId int,song text,userId int

artist            object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
sessionId          int64
song              object
userId             int64
dtype: object

array([False])

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [12]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
import cassandra
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [13]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION ={'class': 'SimpleStrategy','replication_factor': 1}"""
)
except Exception as e: 
    print(e)

#### Set Keyspace

In [14]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e: 
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [28]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during
## sessionId = 338, and itemInSession = 4

try:
    session.execute(""" CREATE TABLE IF NOT EXISTS logtable(artist text,firstName text,gender text,itemInSession text,lastName text,
length float, level text,location text,sessionId text,song text,userId text,PRIMARY KEY(sessionId,itemInSession))""")
except Exception as e:
    print(e)
                    

In [26]:
try:
    session.execute("""DROP TABLE logtable""")
except Exception as e:
    print(e)

In [29]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO logtable(artist,firstName,gender,itemInSession,lastName,
        length, level,location,sessionId,song,userId)"""
        query = query + " VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[1],line[2], line[3],line[4], float(line[5]),line[6], line[7],line[8], line[9],line[10] ))
        #print(line[0], line[1],line[2], line[3],line[4], line[5],line[6], line[7],line[8], line[9],line[10])
        #break

#### Do a SELECT to verify that the data have been inserted into each table

In [30]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

try:
    count = session.execute("select count(*) FROM logtable")
except Exception as e:
    print(e)
for i in count:
    print(i)

Row(count=6820)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [33]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
try:
    count = session.execute("select * FROM logtable where sessionId = '338' and itemInSession = '4'")
except Exception as e:
    print(e)
for i in count:
    print(i)

                    

Row(sessionid='338', iteminsession='4', artist='Faithless', firstname='Ava', gender='F', lastname='Robinson', length=495.30731201171875, level='free', location='New Haven-Milford, CT', song='Music Matters (Mark Knight Dub)', userid='50')


In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

In [39]:
try:
    session.execute(""" CREATE TABLE IF NOT EXISTS logtable1(artist text,firstName text,gender text,itemInSession text,lastName text,
length float, level text,location text,sessionId text,song text,userId text,PRIMARY KEY((userid,sessionId),itemInSession))""")
except Exception as e:
    print(e)

In [37]:
try:
    session.execute("""DROP TABLE logtable1""")
except Exception as e:
    print(e)

### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [41]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = """INSERT INTO logtable1(artist,firstName,gender,itemInSession,lastName,
        length, level,location,sessionId,song,userId)"""
        query = query + " VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0], line[1],line[2], line[3],line[4], float(line[5]),line[6], line[7],line[8], line[9],line[10] ))
        #print(line[0], line[1],line[2], line[3],line[4], line[5],line[6], line[7],line[8], line[9],line[10])
        #break

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()

In [ ]:
 userid = 10, sessionid = 182

In [43]:
try:
    count = session.execute("select * FROM logtable where userid = '10' and sessionid = '182'")
except Exception as e:
    print(e)
for i in count:
    print(i)



Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"
